In [5]:
# Notebook 01: Validación de Conectividad
# **Universidad:** Universidad Nacional Experimental de Guayana (UNEG)  
# **Asignatura:** Sistemas de Bases de Datos II  
# **Proyecto:** Proyecto N2 - Data Pipeline Escalable

# ---

# **Descripción:**  
# Valida la conectividad básica con los servicios de Cassandra y Spark.

In [6]:
from pyspark.sql import SparkSession

# Configuración usando los JARs ya inyectados por el entorno Docker
spark = SparkSession.builder \
    .appName("ValidacionConectividad") \
    .config("spark.cassandra.connection.host", "cassandra") \
    .config("spark.cassandra.connection.localDC", "dc1") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

# 1. Hola Mundo de PySpark
print("--- 1. Validación de PySpark ---")
spark.createDataFrame([("Conexión Exitosa", 1)], ["Mensaje", "ID"]).show()
print("✅ PySpark funciona correctamente.")

--- 1. Validación de PySpark ---
+----------------+---+
|         Mensaje| ID|
+----------------+---+
|Conexión Exitosa|  1|
+----------------+---+

✅ PySpark funciona correctamente.


In [7]:
# 2. Validar conectividad con Cassandra usando cassandra-driver directamente
print("--- 2. Validación de Cassandra (cassandra-driver) ---")

!pip install -q cassandra-driver

from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import DCAwareRoundRobinPolicy

try:
    # Configurar perfil de ejecución con política de balanceo de carga
    # Usamos DCAwareRoundRobinPolicy con local_dc='dc1' como configurado en el docker-compose
    profile = ExecutionProfile(
        load_balancing_policy=DCAwareRoundRobinPolicy(local_dc='dc1')
    )

    # Conectar al cluster especificando el perfil y la versión del protocolo
    # protocol_version=5 es compatible con Cassandra 4.x
    cluster = Cluster(
        ['cassandra'],
        execution_profiles={EXEC_PROFILE_DEFAULT: profile},
        protocol_version=5
    )
    session = cluster.connect()
    
    # Listar keyspaces
    rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces")
    keyspaces = [row.keyspace_name for row in rows]
    
    print(f"✅ Conectado a Cassandra correctamente.")
    print(f"   Keyspaces disponibles: {keyspaces}")
    
    # Verificar si ventas_db existe
    if 'ventas_db' in keyspaces:
        print("   ✅ Keyspace 'ventas_db' encontrado.")
    else:
        print("   ⚠️ Keyspace 'ventas_db' no existe aún. Ejecuta el Notebook 02 para crearlo.")
    
    cluster.shutdown()
except Exception as e:
    print(f"❌ Error al conectar a Cassandra: {e}")

--- 2. Validación de Cassandra (cassandra-driver) ---
✅ Conectado a Cassandra correctamente.
   Keyspaces disponibles: ['ventas_db', 'system_auth', 'system_schema', 'system_distributed', 'system', 'system_traces']
   ✅ Keyspace 'ventas_db' encontrado.


In [8]:
# 3. Validar conector Spark-Cassandra (solo si ventas_db existe)
print("--- 3. Validación del Conector Spark-Cassandra ---")

try:
    # Intentar leer de ventas_db.ventas_crudas si existe
    df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(table="ventas_crudas", keyspace="ventas_db") \
        .load()
    
    count = df.count()
    print(f"✅ Conector Spark-Cassandra funciona correctamente.")
    print(f"   Registros en ventas_crudas: {count:,}")
except Exception as e:
    if "ventas_db" in str(e) or "ventas_crudas" in str(e):
        print("⚠️ La tabla ventas_db.ventas_crudas no existe aún.")
        print("   Esto es normal si no has ejecutado el Notebook 02.")
        print("   El conector Spark-Cassandra está instalado correctamente.")
    else:
        print(f"❌ Error con el conector Spark-Cassandra: {e}")

--- 3. Validación del Conector Spark-Cassandra ---
✅ Conector Spark-Cassandra funciona correctamente.
   Registros en ventas_crudas: 411,474


In [9]:
print("="*60)
print("📋 RESUMEN DE VALIDACIÓN")
print("="*60)
print("✅ PySpark: OK")
print("✅ Cassandra: OK (usa cassandra-driver)")
print("✅ Conector Spark-Cassandra: Instalado")
print("="*60)

📋 RESUMEN DE VALIDACIÓN
✅ PySpark: OK
✅ Cassandra: OK (usa cassandra-driver)
✅ Conector Spark-Cassandra: Instalado
